info


In [2]:
import torch
import torch.nn as nn


c:\Users\Suhyun\.conda\envs\Vision\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class PatchEmbedding(nn.Module):
    def __init__(self, image_size, num_patch, in_dim, latent_dim) -> None:
        super().__init__()

        self.project = nn.Conv2d(in_channels=in_dim, out_channels=latent_dim, kernel_size=image_size, stride=image_size)
        self.cls_token = nn.Parameter(torch.zeros((1,1,latent_dim))) # for concat, one as single patch, latent_dim
        self.pos_embed = nn.Parameter(torch.rand((num_patch + 1, latent_dim))) # for [CLS](+)Patches = 1 + num_path

    def forward(self, img:torch.Tensor):
        # Projection -> Flatten -> add [CLS] token -> Position
        # used in TNT (TokenS-To-TokenS) and CaiT (Cross-Attention Interaction Transformer)

        patches = self.project(img) # (B, C, W, H) -> (B, D, P, P) 
        patches = patches.flatten(2).transpose(1, 2) # (B, num_patch, D)

        patches = torch.concat([self.cls_token, patches], dim=1) # (B, num_patch + 1, D)
        patches = patches + self.pos_embed # (B, num_patch + 1, D) + (num_path + 1, D)...BOARDCAST

        return patches

In [4]:
class PatchEmbedding_ViT(nn.Module):
    def __init__(self, image_size, num_patch, in_dim, latent_dim) -> None:
        super(PatchEmbedding_ViT, self).__init__()

        patch_size = image_size // int(num_patch**0.5) # P*P = H*W / N

        self.project = nn.Linear(in_features= patch_size*patch_size*in_dim, out_features=latent_dim)
        self.cls_token = nn.Parameter(torch.zeros(1,1,latent_dim))
        self.pos_embed = nn.Parameter(torch.zeros(num_patch + 1, latent_dim))

    def forward(self, img):
        # Flatten -> Projection -> Add [CLS] token -> Position
        B, _,_,_ = img.shape

        patches = img.reshape(B, self.num_patch, -1) # (B, C, W, H) -> (B, N, P*P*C ) -- P*P = H*W/N 
        patches = self.project(patches) # (B, N, P*P*C) -> (B, N, D)
        patches = torch.concat([self.cls_token, patches])
        patches = patches + self.pos_embed

        return patches

In [5]:
class Transformer_block(nn.Module):
    def __init__(self, latent_dim, num_heads) -> None:
        super(Transformer_block).__init__()

        self.self_attn = nn.Sequential(
            nn.LayerNorm(latent_dim),
            nn.MultiheadAttention(embed_dim=latent_dim, num_heads=num_heads)
        )

        self.ff = nn.Sequential(
            nn.LayerNorm(latent_dim),
            nn.Linear(in_features=latent_dim, out_features=latent_dim),
            nn.GELU(),
            nn.Linear(in_features=latent_dim, out_features=latent_dim),
        )

    def forward(self, patches):

        residual = patches
        patches = self.self_attn(patches)
        patches = patches + residual

        residual = patches
        patches = self.ff(patches)
        patches = patches + residual

        return patches

In [ ]:
class ViT(nn.Module):
    def __init__(self, img_size, num_patch, input_channel, num_classes, d_model, num_layers, num_heads, is_pretrain=True) -> None:
        super(ViT, self).__init__()
        self.is_pretrain = is_pretrain

        self.patch_emb = PatchEmbedding(image_size=img_size, num_patch=num_patch, in_dim=input_channel, latent_dim=d_model)
        self.transformer_encoder = nn.Sequential(
            *[Transformer_block(latent_dim=d_model, num_heads=num_heads) for _ in range(num_layers)]
        )
        
        if self.is_pretrain:
            self.pretrain_mlp = nn.Sequential(
                nn.LayerNorm(d_model),
                nn.Linear(in_features=d_model, out_features=d_model),
                nn.GELU(),
                nn.Linear(in_features=d_model, out_features=num_classes)
            )
        else:
            self.fc = nn.Linear(in_features=d_model, out_features=num_classes)

    def forward(self, img):

        patches = self.patch_emb(img)
        patches = self.transformer_encoder(patches)
        patches = patches[0] # extract class embedding

        if self.is_pretrain:
            return self.pretrain_mlp(patches)
        else:
            return self.fc(patches)